In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
PROJECT_ID = os.getenv("PROJECT_ID")
PROJECT_NUMBER = os.getenv("PROJECT_NUMBER")
BUCKET_NAME = os.getenv("BUCKET_NAME")
DATASET_ID = os.getenv("DATASET_ID")
CONNECTION_ID = os.getenv("CONNECTION_ID")
SPEECH_MODEL = os.getenv("SPEECH_MODEL")
GENERATIVE_AI_MODEL = os.getenv("GENERATIVE_AI_MODEL")
DATASET_LOCATION = os.getenv("DATASET_LOCATION")
AUDIO_OBJECT_TABLE_ID = os.getenv("AUDIO_OBJECT_TABLE_ID")
MAIN_ACCOUNT_ID = os.getenv("MAIN_ACCOUNT_ID")
SERVICE_ACCOUNT_KEY_ID = os.getenv("SERVICE_ACCOUNT_KEY_ID")
SERVICE_ACCOUNT_KEY_FILE_PATH = os.getenv("SERVICE_ACCOUNT_KEY_FILE_PATH")

# Quick check
print("Bucket:", BUCKET_NAME)
print("Service Account Key File Path:", SERVICE_ACCOUNT_KEY_FILE_PATH)

In [ ]:
!gcloud auth list

### If main account not logged in use this below 2 steps to continue or other wise go to step 5

In [ ]:
# Cell 1: Authenticate to Google Cloud (manual login)
# Only needed if you are running this locally, not in Colab Enterprise
!gcloud auth login

In [ ]:
!gcloud config set account {MAIN_ACCOUNT_ID}

In [ ]:
# Step 1: Set your project ID
!gcloud config set project {PROJECT_ID}

In [ ]:
!gcloud config get-value project

In [ ]:
# Cell 3: Activate service account using key file
# Make sure you have the JSON key at the specified path
!gcloud auth activate-service-account --key-file=$SERVICE_ACCOUNT_KEY_FILE_PATH

#### Make sure your current service account is activated

In [ ]:
!gcloud auth list

In [ ]:
# Step 2: Create a BigQuery Cloud resource connection
# This allows BigQuery to talk to Vertex AI
!bq mk --connection --location=us \
    --connection_type=CLOUD_RESOURCE {CONNECTION_ID}

In [ ]:
# Step 3: Get the Service Account for the connection
SERVICE_ACCT = !bq show --format=prettyjson --connection us.{CONNECTION_ID} | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print("Service Account for connection:", SERVICE_ACCT_EMAIL)

In [ ]:
!gcloud services enable cloudresourcemanager.googleapis.com

In [ ]:
!gcloud services enable documentai.googleapis.com

In [ ]:
!gcloud services enable aiplatform.googleapis.com

In [ ]:
!gcloud services enable speech.googleapis.com

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
  --member="serviceAccount:{SERVICE_ACCOUNT_KEY_ID}" \
  --role="roles/storage.admin"

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role="roles/speech.client"

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/bigquery.connectionUser'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/documentai.viewer'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role="roles/documentai.admin"

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/storage.admin'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/storage.objectViewer'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/speech.editor'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/aiplatform.user'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/bigquery.jobUser'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{SERVICE_ACCT_EMAIL}" \
    --role='roles/bigquery.dataViewer'

In [ ]:
from google.cloud import storage
import os
from pathlib import Path
from google.oauth2 import service_account

# If your notebook is at speak/notebooks/, and the credentials live at speak/credentials/hack36.json

service_account_file = (Path.cwd() / SERVICE_ACCOUNT_KEY_FILE_PATH).resolve()

# sanity check
if not service_account_file.exists():
    raise FileNotFoundError(f"Service account file not found: {service_account_file}")

# create credentials
credentials = service_account.Credentials.from_service_account_file(
    str(service_account_file),
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

storage_client = storage.Client(credentials=credentials, project=credentials.project_id)

def create_gcs_bucket():
    bucket_name = BUCKET_NAME
    bucket = storage_client.bucket(bucket_name)
    if not bucket.exists():
        storage_client.create_bucket(bucket, location=DATASET_LOCATION)
        print(f"✅ Created bucket {bucket_name}")
    else:
        print(f"ℹ️ Bucket {bucket_name} already exists")
    return bucket

create_gcs_bucket()

In [ ]:
# Grant IAM permission to the service account on your GCS bucket
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT_KEY_ID}:roles/storage.objectViewer gs://{BUCKET_NAME}

In [ ]:
# Step 5: Wait for IAM propagation
import time
print("Waiting 60 seconds for IAM propagation...")
time.sleep(60)
print("Setup complete. You can now create remote models pointing to Vertex AI.")

In [ ]:
# Cell 6: Optional: verify permissions and account
!gcloud auth list

In [ ]:
# !gcloud components install alpha --quiet

In [ ]:
# !gcloud alpha ml speech recognizers list \
#     --project=hackathon-project-472403 \
#     --location=us-central1

## Cleaning Up
### To clean up all Google Cloud resources used in this project

In [ ]:
!gcloud projects get-iam-policy {PROJECT_ID}

In [ ]:
# # Delete the BigQuery tables
# ! bq rm --table -f {PROJECT_ID}:{DATASET_ID}.{AUDIO_OBJECT_TABLE_ID}

# # Delete the remote model
# ! bq rm --model -f {PROJECT_ID}:{DATASET_ID}.{SPEECH_MODEL}

# # Delete the remote connection
# ! bq rm --connection --project_id={PROJECT_ID} --location=us {CONNECTION_ID}

# # Delete the BigQuery dataset
# ! bq rm -r -f {PROJECT_ID}:{DATASET_ID}

In [ ]:
# gcloud auth application-default revoke

# gcloud init

# 1] Re-initialize this configuration [default] with new settings

# Login 

# choose created project

# gcloud auth application-default login

# gcloud cheat-sheet 

